In [130]:
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool

from dotenv import dotenv_values

from IPython.display import display

In [131]:
env_values = dotenv_values('.env.local')

DATABASE_URL = env_values.get('DATABASE_URL')
OUTCOME = env_values.get('OUTCOME')

In [132]:
VFT_ENGINE = create_engine(DATABASE_URL,
    poolclass=NullPool,
    future=True)

In [133]:
def getEntity(type, id):
    entities = pd.read_sql(
        '''
        select 
            e.id, e.nome
        from {type} e
        where e.id = {id}
    '''.format(type=type,id=id),
        VFT_ENGINE
    )
    
    return entities.iloc[0]

def getInitiative(id):
    entities = pd.read_sql(
        '''
        select p_i.id, p_i.name as nome, (p_rasc.id is not null) as cadastrado
            from projetos_indicados p_i 
        left join projetos_rascunho p_rasc on p_rasc.id = p_i.projeto_id
        where p_i.id = {id}
    '''.format(id=id),
        VFT_ENGINE
    )
    
    return entities.iloc[0]    

In [134]:
items = pd.read_sql(
    '''
    select 
        p.id, p.nome, pr."data" 
    from projetos_relacoes pr
    inner join projetos_rascunho pr2 on pr2.id = pr.projeto_rascunho_id 
    inner join projetos p on p.id = pr2.projeto_id 
    order by p.nome
''',
    VFT_ENGINE
)

In [ ]:
relations = []

for index, row in items.iterrows():
    # print(f"{row['id']}:{row['nome']} - {json.dumps(row['data'], indent=2)}")
    
    # get politics
    for item in row['data']['politicas']:
        politica = getEntity('politicas',item['politica_id'])
        
        types = []
        for type in item['type']:
            types.append(type['title'])
        
        if len(item['other_type']) > 0:
            types.append(item['other_type'])
            
        relations.append([row['id'],row['nome'],'politica', str(politica['id']), politica['nome'], ', '.join(types)])
        
        
    # get receives
    for target in [['recebe_apoio','recebe'],['oferece_apoio','oferece']]:
        for item in row['data'][target[0]]:
            intituicao = getEntity('instituicoes',item['instituicao_id'])
            
            projeto_nome = None            
            registered = False
            if item['projeto_indicado_id'] is not None:
                projeto = getInitiative(item['projeto_indicado_id'])
                projeto_nome = projeto['nome']
                registered = projeto['cadastrado']
            
            types = []
            for type in item['type']:
                types.append(type['title'])
            
            if len(item['other_type']) > 0:
                types.append(item['other_type'])
                
            inst_texto = intituicao['nome']
            if projeto_nome is not None and len(projeto_nome) > 0:
                reg = ""
                if registered:
                    reg = ' - com cadastro'
                inst_texto += f" (projeto: {projeto_nome}{reg})"
                
            relations.append([row['id'],row['nome'],target[1], str(intituicao['id']), inst_texto, ', '.join(types)])
        
    
relations_dF = pd.DataFrame(relations, columns=["ID.1", "Nome.1","TIPO", "ID.2", "Nome.2", "Motivo"])

if OUTCOME == 'screen':
    display(relations_dF)
else: 
    relations_dF.to_excel("./data/conexoes_pppzcm.xlsx")
    print('DataFrame is written to Excel File successfully.')